# Math Question Answer Verification Competition

## Starter Code

Borrowed from [official Unsloth implementation](https://colab.research.google.com/drive/1Ys44kVvmeZtnICzWz0xgpRnrIOjZAuxp?usp=sharing#scrollTo=MKX_XKs_BNZR)

In [3]:
# %%capture
# This cell will take time
!pip install unsloth
# Also get the latest nightly Unsloth!
!pip uninstall unsloth -y && pip install --upgrade --no-cache-dir "unsloth[colab-new] @ git+https://github.com/unslothai/unsloth.git"

  Using cached unsloth-2024.11.7-py3-none-any.whl.metadata (59 kB)
  Using cached unsloth_zoo-2024.11.5-py3-none-any.whl.metadata (16 kB)
  Using cached xformers-0.0.28.post3-cp310-cp310-manylinux_2_28_x86_64.whl.metadata (1.0 kB)
  Using cached bitsandbytes-0.44.1-py3-none-manylinux_2_24_x86_64.whl.metadata (3.5 kB)
  Using cached triton-3.1.0-cp310-cp310-manylinux_2_17_x86_64.manylinux2014_x86_64.whl.metadata (1.3 kB)
  Using cached tyro-0.8.14-py3-none-any.whl.metadata (8.4 kB)
  Using cached datasets-3.1.0-py3-none-any.whl.metadata (20 kB)
  Using cached trl-0.12.1-py3-none-any.whl.metadata (10 kB)
  Using cached protobuf-3.20.3-cp310-cp310-manylinux_2_12_x86_64.manylinux2010_x86_64.whl.metadata (679 bytes)
  Using cached hf_transfer-0.1.8-cp310-cp310-manylinux_2_17_x86_64.manylinux2014_x86_64.whl.metadata (1.7 kB)
  Using cached dill-0.3.8-py3-none-any.whl.metadata (10 kB)
  Using cached xxhash-3.5.0-cp310-cp310-manylinux_2_17_x86_64.manylinux2014_x86_64.whl.metadata (12 kB)
  Usi

Found existing installation: unsloth 2024.11.7
Uninstalling unsloth-2024.11.7:
  Successfully uninstalled unsloth-2024.11.7
  Cloning https://github.com/unslothai/unsloth.git to /tmp/pip-install-4hl_inj8/unsloth_525b10b9d9dd4c9fb2fe528e561bf3d5
  Running command git clone --filter=blob:none --quiet https://github.com/unslothai/unsloth.git /tmp/pip-install-4hl_inj8/unsloth_525b10b9d9dd4c9fb2fe528e561bf3d5
  Resolved https://github.com/unslothai/unsloth.git to commit f26d4e739ed507de7a9088da53d10fd02f58d160
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
  Created wheel for unsloth: filename=unsloth-2024.11.7-py3-none-any.whl size=163138 sha256=bb72fa1becb9255ea0c0648fbd2211d366816dc3fc7cddb482dded8e67861491
  Stored in directory: /tmp/pip-ephem-wheel-cache-9cwkvgzx/wheels/ed/d4/e9/76fb290ee3df0a5fc21ce5c2c788e29e9607a2353d8342fd0d
Successfully built unsloth


In [4]:
from unsloth import FastLanguageModel
import torch
max_seq_length = 2048 # Choose any
dtype = None # None for auto detection. Float16 for Tesla T4, V100, Bfloat16 for Ampere+
load_in_4bit = True # Use 4bit quantization to reduce memory usage. Can be False.


🦥 Unsloth: Will patch your computer to enable 2x faster free finetuning.


In [5]:
model, tokenizer = FastLanguageModel.from_pretrained(
    model_name = "unsloth/Meta-Llama-3.1-8B",
    max_seq_length = max_seq_length,
    dtype = dtype,
    load_in_4bit = load_in_4bit,
)

==((====))==  Unsloth 2024.11.7: Fast Llama patching. Transformers = 4.46.2.
   \\   /|    GPU: NVIDIA A100-SXM4-40GB. Max memory: 39.564 GB. Platform = Linux.
O^O/ \_/ \    Pytorch: 2.5.1+cu124. CUDA = 8.0. CUDA Toolkit = 12.4.
\        /    Bfloat16 = TRUE. FA [Xformers = 0.0.28.post3. FA2 = False]
 "-____-"     Free Apache license: http://github.com/unslothai/unsloth
Unsloth: Fast downloading is enabled - ignore downloading bars which are red colored!


model.safetensors:   0%|          | 0.00/5.70G [00:00<?, ?B/s]

generation_config.json:   0%|          | 0.00/230 [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/50.6k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/9.09M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/345 [00:00<?, ?B/s]

## Load model and wrap with LoRA adapters

In [6]:
model = FastLanguageModel.get_peft_model(
    model,
    r = 16, # Choose any number > 0 ! Suggested 8, 16, 32, 64, 128
    target_modules = ["q_proj", "k_proj", "v_proj", "o_proj",
                      "gate_proj", "up_proj", "down_proj",],
    lora_alpha = 16,
    lora_dropout = 0, # Supports any, but = 0 is optimized
    bias = "none",    # Supports any, but = "none" is optimized
    # [NEW] "unsloth" uses 30% less VRAM, fits 2x larger batch sizes!
    use_gradient_checkpointing = "unsloth", # True or "unsloth" for very long context
    random_state = 3407,
    use_rslora = False,  # We support rank stabilized LoRA
    loftq_config = None, # And LoftQ
)

Unsloth 2024.11.7 patched 32 layers with 32 QKV layers, 32 O layers and 32 MLP layers.


## Competition dataset

In [7]:
# download and load competition dataset

from datasets import load_dataset
dataset = load_dataset("ad6398/nyu-dl-teach-maths-comp")
# print and see dataset
dataset

README.md:   0%|          | 0.00/2.09k [00:00<?, ?B/s]

train-00000-of-00002.parquet:   0%|          | 0.00/195M [00:00<?, ?B/s]

train-00001-of-00002.parquet:   0%|          | 0.00/195M [00:00<?, ?B/s]

test-00000-of-00001.parquet:   0%|          | 0.00/3.65M [00:00<?, ?B/s]

Generating train split:   0%|          | 0/1000000 [00:00<?, ? examples/s]

Generating test split:   0%|          | 0/10000 [00:00<?, ? examples/s]

DatasetDict({
    train: Dataset({
        features: ['question', 'is_correct', 'answer', 'solution'],
        num_rows: 1000000
    })
    test: Dataset({
        features: ['question', 'is_correct', 'answer', 'solution'],
        num_rows: 10000
    })
})

In [8]:
# prompt = """You are a great mathematician and you are tasked with finding if an answer to a given maths question is correct or not. Yout response should be 'True' if correct, otherwise 'False'. Below is Question and Answer.



# ### Question:
# {}

# ### Answer:
# {}

# ### Explainaition

# ### Output:
# {}"""
prompt = """You are a knowledgeable mathematician. Evaluate the correctness of the answer to the question. The solution is provided to help you verify the answer logically.
### Question:
{}
### Answer:
{}
### Solution:
{}
### Output:
{}"""

EOS_TOKEN = tokenizer.eos_token # Must add EOS_TOKEN
def formatting_prompts_func(examples):
    question = examples["question"]
    ans       = examples["answer"]
    solution = examples["solution"]
    output      = examples["is_correct"]

    texts = []
    for instruction, input, hint, output in zip(question, ans, solution, output):
        # Must add EOS_TOKEN, otherwise your generation will go on forever!
        text = prompt.format(instruction, input, hint, output) + EOS_TOKEN
        texts.append(text)
    return { "text" : texts, }




In [9]:
# Process the training dataset and generate prompt for each datapoint

train_dataset = dataset['train'].map(formatting_prompts_func, batched = True,)

Map:   0%|          | 0/1000000 [00:00<?, ? examples/s]

In [10]:
#print a smaple training example
train_dataset['text'][0]

"You are a knowledgeable mathematician. Evaluate the correctness of the answer to the question. The solution is provided to help you verify the answer logically.\n### Question:\nWhat is the radius of the circle inscribed in triangle $ABC$ if $AB = 22, AC=12,$ and $BC=14$? Express your answer in simplest radical form.\n### Answer:\n3.16227766016838\n### Solution:\nThe circle is inscribed in a triangle, and we know the sides of the triangle.\nTo use the inradius formula, we need to know the area of the triangle.\nWe can use Heron's formula to calculate the area.\n<llm-code>\nimport math\nfrom sympy import *\n\nAB, AC, BC = 22, 12, 14\n\n# Calculate the semiperimeter and area using Heron's formula\ns = (AB + AC + BC) / 2\nK = sqrt(s * (s - AB) * (s - AC) * (s - BC))\n\nprint(K)\n</llm-code>\n<llm-code-output>\n75.8946638440411\n</llm-code-output>\nLet's now use the formula for the radius of the inscribed circle.\n<llm-code>\nr = K / s\nprint(r)\n</llm-code>\n<llm-code-output>\n3.162277660

In [11]:
# 划分训练集和验证集
dataset_split = dataset['train'].train_test_split(test_size=0.01)
train_dataset = dataset_split['train']
eval_dataset = dataset_split['test']

# 重新生成 'text' 字段
train_dataset = train_dataset.map(formatting_prompts_func, batched=True)
eval_dataset = eval_dataset.map(formatting_prompts_func, batched=True)

# 检查生成的字段是否正确
print("train text:", train_dataset.column_names)
print("validate text:", eval_dataset.column_names)


Map:   0%|          | 0/990000 [00:00<?, ? examples/s]

Map:   0%|          | 0/10000 [00:00<?, ? examples/s]

train text: ['question', 'is_correct', 'answer', 'solution', 'text']
validate text: ['question', 'is_correct', 'answer', 'solution', 'text']


## SFT

In [12]:
import logging
logging.basicConfig(level=logging.INFO)

In [13]:
from trl import SFTTrainer
from transformers import TrainingArguments
from unsloth import is_bfloat16_supported


# 使用验证集的子集

training_args = TrainingArguments(
    per_device_train_batch_size = 4,
    gradient_accumulation_steps = 4,  # 增加梯度累积步数，提升等效批次
    warmup_steps=100,                # 增加 warmup 步数，适应更长的训练
    max_steps=3000,                  # 增加总训练步数
    learning_rate=2e-4,
    # lr_scheduler_type="cosine",  # 使用cosine学习率调度器
    fp16=not is_bfloat16_supported(),
    bf16=is_bfloat16_supported(),
    logging_steps=10,
    eval_steps=200,
    evaluation_strategy="steps",  # 添加此行
    logging_strategy="steps",     # 添加此行
    optim="adamw_8bit",
    weight_decay=0.02,
    lr_scheduler_type="cosine",
    seed=3407,
    output_dir="outputs",
    report_to="none"
)

trainer = SFTTrainer(
    model=model,
    tokenizer=tokenizer,
    train_dataset=train_dataset,
    eval_dataset=eval_dataset,  # 添加验证集，监控模型效果
    dataset_text_field="text",
    max_seq_length=max_seq_length,
    dataset_num_proc=4,
    packing=False,
    args=training_args
)

/usr/local/lib/python3.10/dist-packages/transformers/training_args.py:1568: FutureWarning: `evaluation_strategy` is deprecated and will be removed in version 4.46 of 🤗 Transformers. Use `eval_strategy` instead
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/transformers/training_args.py:1568: FutureWarning: `evaluation_strategy` is deprecated and will be removed in version 4.46 of 🤗 Transformers. Use `eval_strategy` instead
  warnings.warn(


Map (num_proc=4):   0%|          | 0/990000 [00:00<?, ? examples/s]

Map (num_proc=4):   0%|          | 0/10000 [00:00<?, ? examples/s]

max_steps is given, it will override any value given in num_train_epochs


In [14]:
trainer_stats = trainer.train()

==((====))==  Unsloth - 2x faster free finetuning | Num GPUs = 1
   \\   /|    Num examples = 990,000 | Num Epochs = 1
O^O/ \_/ \    Batch size per device = 4 | Gradient Accumulation steps = 4
\        /    Total batch size = 16 | Total steps = 3,000
 "-____-"     Number of trainable parameters = 41,943,040


Step,Training Loss,Validation Loss
200,0.706100,0.727116
400,0.708300,0.706040
600,0.720900,0.690252
800,0.704800,0.675356
1000,0.648800,0.659033
1200,0.606200,0.645290
1400,0.647100,0.629300
1600,0.655200,0.614311
1800,0.604700,0.600062
2000,0.550700,0.588825


In [ ]:
import random
import torch
from sklearn.metrics import accuracy_score
import pandas as pd
from unsloth import FastLanguageModel

# 启用快速推理模式
FastLanguageModel.for_inference(model)

# 1. 随机抽取1,000条样本
sample_size = 1000
eval_sample = eval_dataset.select(random.sample(range(len(eval_dataset)), sample_size))

# 2. 设置批量大小
batch_size = 32
predictions = []
true_labels = eval_sample["is_correct"]

# 3. 批量处理抽取的样本
for i in range(0, len(eval_sample), batch_size):
    batch = eval_sample.select(range(i, min(i + batch_size, len(eval_sample))))
    input_prompts = [
        f"### Question:\n{item['question']}\n\n### Answer:\n{item['answer']}\n\n### Output:\n"
        for item in batch
    ]

    # 使用 tokenizer 处理批次输入
    inputs = tokenizer(input_prompts, return_tensors="pt", padding=True, truncation=True, max_length=512)

    # 将输入数据移动到 GPU（如果有）
    inputs = {k: v.to("cuda") for k, v in inputs.items()} if torch.cuda.is_available() else inputs

    # 批量生成预测结果
    outputs = model.generate(inputs["input_ids"], attention_mask=inputs["attention_mask"], max_new_tokens=64)
    responses = tokenizer.batch_decode(outputs, skip_special_tokens=True)

    # 将每个响应转化为 True 或 False
    batch_predictions = [response.strip() == "True" for response in responses]
    predictions.extend(batch_predictions)

# 4. 计算抽样验证集的准确率
accuracy = accuracy_score(true_labels, predictions)
print(f"抽样验证集准确率: {accuracy:.2f}")


抽样验证集准确率: 0.60


In [ ]:
from tqdm import tqdm
import pandas as pd
from unsloth import FastLanguageModel

# 确保已加载测试集
test_dataset = dataset['test']

# 启用快速推理模式
FastLanguageModel.for_inference(model)

# 初始化预测结果列表
test_predictions = []

# 设置批量大小
batch_size = 32

# 使用 tqdm 包装循环以显示进度条
for i in tqdm(range(0, len(test_dataset), batch_size), desc="Processing test dataset"):
    # 分批获取测试数据
    batch = test_dataset.select(range(i, min(i + batch_size, len(test_dataset))))
    input_prompts = [
        f"### Question:\n{item['question']}\n\n### Answer:\n{item['answer']}\n\n### Output:\n"
        for item in batch
    ]

    # 使用 tokenizer 对批次进行处理
    inputs = tokenizer(input_prompts, return_tensors="pt", padding=True, truncation=True, max_length=512)

    # 将输入数据移动到 GPU（如果可用）
    inputs = {k: v.to("cuda") for k, v in inputs.items()} if torch.cuda.is_available() else inputs

    # 批量生成预测结果
    outputs = model.generate(inputs["input_ids"], attention_mask=inputs["attention_mask"], max_new_tokens=64)
    responses = tokenizer.batch_decode(outputs, skip_special_tokens=True)

    # 解析每个响应并存储到列表
    for j, response in enumerate(responses):
        is_correct = response.strip() == "True"  # 检查预测结果是否为“True”
        test_predictions.append({"ID": i + j, "is_correct": is_correct})

# 将预测结果保存为符合竞赛格式的CSV文件
submission_df = pd.DataFrame(test_predictions)
submission_df.to_csv("submission_competition.csv", index=False)

print("测试集预测结果已生成并保存为 submission_competition.csv 文件。")


Processing test dataset: 100%|██████████| 313/313 [29:27<00:00,  5.65s/it]

测试集预测结果已生成并保存为 submission_competition.csv 文件。


In [ ]:
from google.colab import files
files.download("submission_competition.csv")


<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

## inference

In [16]:
from tqdm import tqdm
from unsloth import FastLanguageModel
test_dataset = dataset['test']
# 启用快速推理模式
FastLanguageModel.for_inference(model)

# 遍历前20个样本并进行推理
num_samples = 100
results = []

for i in tqdm(range(num_samples), desc="Running inference for 20 samples"):
    sample_ques = test_dataset['question'][i]
    sample_ans = test_dataset['answer'][i]
    sample_sol = test_dataset['solution'][i]

    # 构造输入提示
    input_prompt = prompt.format(sample_ques, sample_ans, sample_sol, "")

    # 编码输入提示
    inputs = tokenizer([input_prompt], return_tensors="pt").to("cuda")
    input_shape = inputs['input_ids'].shape
    input_token_len = input_shape[1]

    # 生成响应，并限制生成长度
    outputs = model.generate(**inputs, max_new_tokens=5, use_cache=True, eos_token_id=tokenizer.eos_token_id)
    response = tokenizer.decode(outputs[0][input_token_len:], skip_special_tokens=True).strip()

    # 判断生成的响应是否为“True”或“False”
    if 'true' in response.lower():
        is_correct = 'True'
    elif 'false' in response.lower():
        is_correct = 'False'
    else:
        is_correct = "missing"  # 若生成的结果无法判断

    # 仅保存ID和判断结果
    results.append({'ID': i, 'is_correct': is_correct})

# 打印出所有结果
for result in results:
    print(result)


Running inference for 20 samples: 100%|██████████| 100/100 [00:23<00:00,  4.27it/s]

{'ID': 0, 'is_correct': 'True'}
{'ID': 1, 'is_correct': 'False'}
{'ID': 2, 'is_correct': 'True'}
{'ID': 3, 'is_correct': 'True'}
{'ID': 4, 'is_correct': 'False'}
{'ID': 5, 'is_correct': 'False'}
{'ID': 6, 'is_correct': 'False'}
{'ID': 7, 'is_correct': 'False'}
{'ID': 8, 'is_correct': 'False'}
{'ID': 9, 'is_correct': 'False'}
{'ID': 10, 'is_correct': 'False'}
{'ID': 11, 'is_correct': 'False'}
{'ID': 12, 'is_correct': 'False'}
{'ID': 13, 'is_correct': 'False'}
{'ID': 14, 'is_correct': 'True'}
{'ID': 15, 'is_correct': 'True'}
{'ID': 16, 'is_correct': 'False'}
{'ID': 17, 'is_correct': 'False'}
{'ID': 18, 'is_correct': 'False'}
{'ID': 19, 'is_correct': 'False'}
{'ID': 20, 'is_correct': 'False'}
{'ID': 21, 'is_correct': 'True'}
{'ID': 22, 'is_correct': 'False'}
{'ID': 23, 'is_correct': 'True'}
{'ID': 24, 'is_correct': 'True'}
{'ID': 25, 'is_correct': 'True'}
{'ID': 26, 'is_correct': 'False'}
{'ID': 27, 'is_correct': 'False'}
{'ID': 28, 'is_correct': 'False'}
{'ID': 29, 'is_correct': 'True'}


In [18]:
from tqdm import tqdm
from unsloth import FastLanguageModel

# 启用快速推理模式
FastLanguageModel.for_inference(model)

# 设置批量大小和样本总数
batch_size = 32
num_samples = len(test_dataset) # 总共处理的样本数量

results = []

# 逐批处理样本
for start_idx in tqdm(range(0, num_samples, batch_size), desc="Running inference in batches"):
    # 获取当前批次的样本
    end_idx = min(start_idx + batch_size, num_samples)
    batch_samples = test_dataset['question'][start_idx:end_idx]
    batch_answers = test_dataset['answer'][start_idx:end_idx]
    batch_solution = test_dataset['solution'][start_idx:end_idx]

    # 构造输入提示
    input_prompts = [prompt.format(q, a, s, "") for q, a, s in zip(batch_samples, batch_answers, batch_solution)]

    # 编码输入提示
    inputs = tokenizer(input_prompts, return_tensors="pt", padding=True, truncation=True).to("cuda")
    input_shape = inputs['input_ids'].shape[1]

    # 执行批量生成
    outputs = model.generate(**inputs, max_new_tokens=5, use_cache=True, eos_token_id=tokenizer.eos_token_id)

    # 解码并解析批量生成的响应
    responses = tokenizer.batch_decode(outputs[:, input_shape:], skip_special_tokens=True)

    for i, response in enumerate(responses):
        # 判断生成的响应是否包含“True”或“False”
        if 'true' in response.lower():
            is_correct = 'True'
        elif 'false' in response.lower():
            is_correct = 'False'
        else:
            is_correct = "无法判断"  # 若生成的结果无法判断

        # 保存结果
        results.append({'ID': start_idx + i, 'is_correct': is_correct})

# 打印出所有结果
# for result in results:
#     print(result)


Running inference in batches: 100%|██████████| 313/313 [09:36<00:00,  1.84s/it]


In [19]:
import pandas as pd

# 将结果保存为 CSV 文件
df = pd.DataFrame(results)
df.to_csv('test_set_predictions.csv', index=False)

print("全量测试集推理完成，结果已保存为 'test_set_predictions.csv'")


全量测试集推理完成，结果已保存为 'test_set_predictions.csv'


## saving model

In [20]:
model.save_pretrained("lora_model") # Local saving
tokenizer.save_pretrained("lora_model")

('lora_model/tokenizer_config.json',
 'lora_model/special_tokens_map.json',
 'lora_model/tokenizer.json')

In [21]:
if True:
    from unsloth import FastLanguageModel
    model, tokenizer = FastLanguageModel.from_pretrained(
        model_name = "lora_model", # YOUR MODEL YOU USED FOR TRAINING
        max_seq_length = max_seq_length,
        dtype = dtype,
        load_in_4bit = load_in_4bit,
    )
    FastLanguageModel.for_inference(model) # Enable native 2x faster inference


==((====))==  Unsloth 2024.11.7: Fast Llama patching. Transformers = 4.46.2.
   \\   /|    GPU: NVIDIA A100-SXM4-40GB. Max memory: 39.564 GB. Platform = Linux.
O^O/ \_/ \    Pytorch: 2.5.1+cu124. CUDA = 8.0. CUDA Toolkit = 12.4.
\        /    Bfloat16 = TRUE. FA [Xformers = 0.0.28.post3. FA2 = False]
 "-____-"     Free Apache license: http://github.com/unslothai/unsloth
Unsloth: Fast downloading is enabled - ignore downloading bars which are red colored!


In [22]:
!zip -r lora_model.zip lora_model


  adding: lora_model/ (stored 0%)
  adding: lora_model/tokenizer.json (deflated 85%)
  adding: lora_model/special_tokens_map.json (deflated 71%)
  adding: lora_model/adapter_model.safetensors (deflated 7%)
  adding: lora_model/adapter_config.json (deflated 54%)
  adding: lora_model/tokenizer_config.json (deflated 96%)
  adding: lora_model/README.md (deflated 66%)
